## Importing Packages

In [121]:
import os
import sys
import string
from urllib.request import Request, urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import time
import datetime
import ssl
import numpy as np
import pandas as pd
import traceback
import csv
ssl._create_default_https_context = ssl._create_unverified_context
import json
import subprocess
import glob
import requests
import shutil
from tqdm import tqdm

### downloading source content

In [122]:
def get_source_content(url):
    response = requests.get(url)

    if response.status_code == 200:
        return response.content
    else:
        return None

### find all the line numbers that the functions begins


In [123]:
# find all the line numbers that the functions begins
def get_line_numbers(filename,lang_type):
    # found = False
    #cmd = "ctags -x --c-kinds=fp " + filename + " | grep " + funcname
    cmd = "ctags -x --"+lang_type+"-kinds=f " + filename

    output = subprocess.getoutput(cmd)
    lines = output.splitlines()
    line_nums = []
    for line in lines:
        line = line.split(" ")
        char = list(filter(None, line))
        line_num = char[2]
        line_nums.append(int(line_num))
    return line_nums


In [124]:

def process_file(filename, line_num):
    # print("opening " + filename + " on line " + str(line_num))

    code = ""
    cnt_braket = 0
    found_start = False
    found_end = False

    with open(filename, "r") as f:
        for i, line in enumerate(f):
            if(i >= (line_num - 1)):
                code += line

                if (not line.startswith("//")) and line.count("{") > 0:
                    found_start = True
                    cnt_braket += line.count("{")

                if (not line.startswith("//")) and line.count("}") > 0:
                    cnt_braket -= line.count("}")

                if cnt_braket == 0 and found_start == True:
                    found_end = True
                    return code, i+1

In [125]:
def get_diff_num(filename):
    diff_start_lines = []
    with open(filename, "r") as patch:
        for i, line in enumerate(patch):
            if line.startswith("@@ "):
                if not i == 0:
                    diff_start_lines.append(i)
        diff_start_lines.append(i+1)
    return diff_start_lines

def get_enumerate(filename):
    patch = open(filename, "r")
    return enumerate(patch)

In [126]:

def get_diff_information(filename, diff_start_lines):
    block_num = 0
    archor = []
    count = 0
    for diff_start_line in diff_start_lines:
        # reset some values
        count += 1
        minus_count = 0
        plus_count = 0
        before = None
        after = None
        start_line_num = None
        minus_pos = []
        plus_pos = []
        patch = []
        for j, l in get_enumerate(filename):
            if count == len(diff_start_lines):
                if j == diff_start_line - 1:
                    patch.append(l)
                    if l.startswith("-"):
                        minus_count += 1
                        minus_pos.append(j - start_line_num)
                    if l.startswith("+"):
                        plus_count += 1
                        plus_pos.append(j - start_line_num)
                    block_num = j
                    before = list(map(int, before))
                    after = list(map(int, after))
                    if len(archor) == 0:
                        archor.append(
                            [before, after, minus_count, plus_count, minus_pos, plus_pos, patch, after[0], minus_count,
                             0])
                    else:
                        diff_value = archor[-1][8] + minus_count
                        insert_after = after[0] + archor[-1][8]
                        last_end = archor[-1][1][0] + archor[-1][1][1] - 1
                        archor.append([before, after, minus_count, plus_count, minus_pos, plus_pos, patch, insert_after,
                                       diff_value, last_end])
                    break

                if block_num <= j < diff_start_line - 1:
                    patch.append(l)
                    if l.startswith("@@ "):
                        start_line_num = j
                        pos = l.find("@@ ")
                        end = l.find(" @@ ")
                        modified = l[pos + 3:end]
                        modified = modified.split(" ")
                        before = modified[0]
                        before = before.replace("-", "")
                        before = before.split(",")
                        after = modified[1]
                        after = after.replace("+", "")
                        after = after.split(",")
                        # now our source files are after-modified

                    if l.startswith("-"):
                        minus_count += 1
                        minus_pos.append(j - start_line_num)

                    if l.startswith("+"):
                        plus_count += 1
                        plus_pos.append(j - start_line_num)
                elif j < block_num:
                    continue
                else:
                    block_num = j
                    before = list(map(int, before))
                    after = list(map(int, after))
                    if len(archor) == 0:
                        archor.append(
                            [before, after, minus_count, plus_count, minus_pos, plus_pos, patch, after[0], minus_count,
                             0])
                    else:
                        diff_value = archor[-1][8] + minus_count
                        insert_after = after[0] + archor[-1][8]
                        last_end = archor[-1][1][0] + archor[-1][1][1] - 1
                        archor.append([before, after, minus_count, plus_count, minus_pos, plus_pos, patch, insert_after,
                                       diff_value, last_end])
                    break
            else:
                if block_num <= j < diff_start_line:
                    patch.append(l)
                    if l.startswith("@@ "):
                        start_line_num = j
                        pos = l.find("@@ ")
                        end = l.find(" @@ ")
                        modified = l[pos + 3:end]
                        modified = modified.split(" ")
                        before = modified[0]
                        before = before.replace("-", "")
                        before = before.split(",")
                        after = modified[1]
                        after = after.replace("+", "")
                        after = after.split(",")
                        # now our source files are after-modified

                    if l.startswith("-"):
                        minus_count += 1
                        minus_pos.append(j - start_line_num)

                    if l.startswith("+"):
                        plus_count += 1
                        plus_pos.append(j - start_line_num)
                elif j < block_num:
                    continue
                else:
                    block_num = j
                    before = list(map(int, before))
                    after = list(map(int, after))
                    if len(archor) == 0:
                        archor.append(
                            [before, after, minus_count, plus_count, minus_pos, plus_pos, patch, after[0], minus_count,
                             0])
                    else:
                        diff_value = archor[-1][8] + minus_count
                        insert_after = after[0] + archor[-1][8]
                        last_end = archor[-1][1][0] + archor[-1][1][1] - 1
                        archor.append([before, after, minus_count, plus_count, minus_pos, plus_pos, patch, insert_after,
                                       diff_value, last_end])
                    break

    return archor

# Vulnerable functions

In [127]:
def test(file_content):
    lines = file_content.splitlines()

    vulnerable_function = []
    skip_next_line_vuln = False

    for line in lines:
        # Skip lines after `//fix_flaw_line_below:`
        if "//fix_flaw_line_below:" in line:
            skip_next_line_vuln = True
            continue
        if skip_next_line_vuln:
            skip_next_line_vuln = False
            continue

        # Skip comments
        if line.strip().startswith("//"):
            continue

        # Add the line to the vulnerable function
        vulnerable_function.append(line)

    # Join the lines to get the final function code as a string
    vulnerable_function_code = "\n".join(vulnerable_function)
    # print(vulnerable_function_code)

    return vulnerable_function_code




### Non vulnerable functions

In [128]:
def test2(file_content):
    lines = file_content.splitlines()

    non_vulnerable_function = []
    skip_next_line_vuln = False
    uncomment_next_line_fix = False

    for line in lines:
        # Check if the line starts with `//flaw_line_below:`
        if line.strip().startswith("//flaw_line_below:"):
            skip_next_line_vuln = True
            continue

        # If the previous line was `//flaw_line_below:`, skip this line
        if skip_next_line_vuln:
            skip_next_line_vuln = False
            continue

        # Check if the line starts with `//fix_flaw_line_below:`
        if line.strip().startswith("//fix_flaw_line_below:"):
            uncomment_next_line_fix = True
            continue

        # If the previous line was `//fix_flaw_line_below:`, uncomment this line
        if uncomment_next_line_fix:
            uncomment_next_line_fix = False
            non_vulnerable_function.append(line.lstrip("//").strip() + "\n")
            continue

        # Add the line to the non-vulnerable function
        non_vulnerable_function.append(line)

    # Join the lines to get the final function code as a string
    non_vulnerable_function_code = "\n".join(non_vulnerable_function)

    return non_vulnerable_function_code


In [129]:
def process_patch_file(file_path):
    non_vuln_lines = []
    vuln_lines = []

    with open(file_path, 'r') as file:
        for line in file:
            line = line.rstrip()  # Remove any trailing newline or spaces

            if line.startswith('+'):
                non_vuln_lines.append(line[1:].strip())  # Remove the '+' symbol and any extra whitespace
            elif line.startswith('-'):
                vuln_lines.append(line[1:].strip())  # Remove the '-' symbol and any extra whitespace

    return non_vuln_lines, vuln_lines


In [130]:
def categorize_code_lines(code):
    vuln_lines = []
    non_vuln_lines = []
    
    # Split the input code into lines
    lines = code.split('\n')
    
    is_vuln_next = False
    is_fix_next = False
    
    for line in lines:
        # Check if the current line indicates the next line is vulnerable
        if "//flaw_line_below" in line:
            is_vuln_next = True
            continue
        # Check if the current line indicates the next line is non-vulnerable
        elif "//fix_flaw_line_below" in line:
            is_fix_next = True
            continue
        
        # If the flag is set, categorize the next line accordingly
        if is_vuln_next:
            vuln_lines.append(line.strip())
            is_vuln_next = False
        elif is_fix_next:
            # Remove leading // and any extra spaces for non-vulnerable lines
            non_vuln_lines.append(line.lstrip('/ ').strip())
            is_fix_next = False

    formatted_vuln_lines = "\n".join(vuln_lines)
    formatted_non_vuln_lines = "\n".join(non_vuln_lines)

    return formatted_vuln_lines, formatted_non_vuln_lines

In [131]:
def generate_line_diff(c_cpp_csv):
    vul_number = 0 
    data=[]

    for index, row in tqdm(c_cpp_csv.iterrows(), total=len(c_cpp_csv)  , desc="Processing function with line extraction..."):
         try:   
            commit_id = row["commit_id"]    # commit id
            diff = row["files_changed"]
            if not (row["cwe_id"] is None) and not (row["cwe_id"] == ""):
                CWE_ID = str(row["cwe_id"])
            else:
                CWE_ID = "others"
            
            files_changed = []     # files changed in this commit
            project = row["project"]     # project name
            for i in diff.split("<_**next**_>"):
                files_changed.append(json.loads(i))

            for file in files_changed:
                file_with_dir = file["filename"]      # file name with full path
                pos = file_with_dir.rfind('/')
                if pos > 0:
                    filename = file_with_dir[pos + 1:]  # file name
                    file_dir = commit_id + "/" + file_with_dir[:pos]
                elif pos == 0:
                    filename = file_with_dir[1:]
                    file_dir = commit_id
                else:
                    filename = file_with_dir
                    file_dir = commit_id
                raw_url=file["raw_url"]    # raw url
                if "patch" in file:
                    patch = file["patch"]
                else:
                    patch = ""
                type_pos = filename.find('.')
                if type_pos > 0:
                    only_name = filename[:type_pos]
                    only_type = filename[type_pos + 1:]
                else:
                    only_name = filename
                    only_type = "not know"
                
                sourcefiles = get_source_content(raw_url)       # get source code from raw url
                if sourcefiles is not None:
                   sourcefiles_str = sourcefiles.decode('utf-8') 

                # TODO: get sourcefiles from local

                if not os.path.exists("patchAll0206/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir):
                    os.makedirs("patchAll0206/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir)
                sourcefile_dir = "patchAll0206/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir + '/' + filename
                # print("sourcefiledir",sourcefile_dir)
                patchfile_dir = "patchAll0206/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir + '/' + only_name + '_' + 'patch.txt'
                # print("patchfile_dir",patchfile_dir)
                with open(sourcefile_dir, "w+") as source_file, open(patchfile_dir, "w+") as patch_file:
                    source_file.write(sourcefiles_str)
                    patch_file.write(patch)
                # TODO: get functions: if vul? not vul?
                if only_type == "c":

                    num = get_diff_num(patchfile_dir)     # get the number of diff blocks

                    archors = get_diff_information(patchfile_dir, num)    # get the diff information

                    block_num = 0
                    block_total = len(archors)
                    for archor in archors:
                        block_num += 1
                        del_line_pos = archor[4]
                        add_line_pos = archor[5]
                        patch_start = int(archor[1][0])
                        patch_lines = int(archor[0][1]) + archor[3]
                        patch_end = patch_start + patch_lines - 1
                        source_end = patch_start + int(archor[1][1]) - 1
                        last_end = archor[9]
                        wrote = False

                        add_patch_file_dir = "patchAll0206/" + only_type + '/' + project + '/' + CWE_ID + '/' + file_dir + '/' + "add_patch_" + filename
                        with open(sourcefile_dir, "r") as before, open(add_patch_file_dir, "a") as after:
                            lines = before.readlines()
                            flen = len(lines)  # the number of lines in the source file
                            for i in range(flen):
                                if last_end - 1 < i <= source_end - 1:
                                    if i == 0:
                                        after.write(lines[i])
                                        continue
                                    if (patch_start - 1 <= i <= source_end - 1):
                                        if wrote == False:
                                            for patch_line in archor[6][1:]:
                                                if patch_line.startswith("+"):
                                                    patch_line = patch_line.replace("+", "//fix_flaw_line_below:\n//",
                                                                                     1)
                                                if patch_line.startswith("-"):
                                                    patch_line = patch_line.replace("-", "//flaw_line_below:\n", 1)
                                                if not patch_line.endswith("\n"):
                                                    patch_line = patch_line + "\n"
                                                after.write(patch_line)
                                            wrote = True
                                    else:
                                        after.write(lines[i])
                                if block_num == block_total and source_end < flen and i > source_end - 1:
                                    after.write(lines[i])

                    line_nums = get_line_numbers(add_patch_file_dir, "c")
                    if len(line_nums) > 0:
                        for line_num in line_nums:
                            code, i = process_file(add_patch_file_dir, line_num)    # get the function code and the end line number
                            if "//flaw_line_below:" in code or "//fix_flaw_line_below:\n//" in code:
                                vuln_code = test(code)
                                non_vuln_code =test2(code)
                                vuln_line, non_vuln_line = categorize_code_lines(code)
                                vul_number += 1
                                split_vul_dir = "./split0206/vul" + '/' + project + '/' + CWE_ID
                                if not os.path.exists(split_vul_dir):
                                    os.makedirs(split_vul_dir)
                                split_vul_file = split_vul_dir + '/' + CWE_ID + "_" + "add_patch_" + str(
                                    i) + "_" + filename
                                with open(split_vul_file, "w+") as vulFun:
                                    vulFun.write(code)
                                    
                                split_vul_dir_0 = "./split0206/vul0" + '/' + project
                                if not os.path.exists(split_vul_dir_0):
                                    os.makedirs(split_vul_dir_0)
                                split_vul_file_0 = split_vul_dir_0 + '/' + CWE_ID + "_""add_patch_" + str(
                                    i) + "_" + filename
                                with open(split_vul_file_0, "w+") as vulFun0:
                                    vulFun0.write(code)
                                
                                cve_id = row["cve_id"]
                                cwe_id = row["cwe_id"]
                                project = row["project"]
                                commit_id = row["commit_id"]
                                ref_link = row["ref_link"]
                                vulnerability_classification = row["vulnerability_classification"]
                                score=row["score"]

                                # result={
                                #     "cve_id":cve_id,
                                #     "cwe_id":cwe_id,
                                #     "commit_link":ref_link,
                                #     "project":project,
                                #     "commit_id":commit_id,
                                #     "score":score,
                                #     "language":only_type,
                                #     "vulnerability_type":vulnerability_classification,
                                #     "filename_with_path":file_with_dir,
                                #     "vuln_line":vuln_line,
                                #     "non_vuln_line":non_vuln_line,
                                #     "vuln_function":vuln_code,
                                #     "non_vuln_function":non_vuln_code,
                                #     "filename":filename,
                                #     "raw_url":raw_url,
                                # }
                                # print("This is result",result)
                                # data.append(result)

                                lines = code.split('\n')

                                # Extract flaw lines and fix flaw lines
                                for i, line in enumerate(lines):
                                    line_before=""
                                    line_after=""
                                    if "//flaw_line_below:" in line:
                                        # assign value to line_before
                                        line_before = lines[i + 1].strip() if i + 1 < len(lines) else ""
                                        print("line_before",line_before)

                                        
                                      
                                     
                                    elif "//fix_flaw_line_below:" in line:
                                        # assign value to line_after
                                        line_after = lines[i + 1].strip() if i + 1 < len(lines) else ""
                                    # print("line_after",line_after)

                                # Ensure both lists have the same length by filling missing entries with "Not Available"
                                # max_length = max(len(flaw_lines), len(fix_flaw_lines))
                                # flaw_lines += ["Not Available"] * (max_length - len(flaw_lines))
                                # fix_flaw_lines += ["Not Available"] * (max_length - len(fix_flaw_lines))
                        
                            else:
                                split_nonevul_dir = "./split0206/nonevul" + '/' + project
                                if not os.path.exists(split_nonevul_dir):
                                    os.makedirs(split_nonevul_dir)
                                split_nonevul_file = split_nonevul_dir + '/' + "add_patch_" + str(i) + "_" + filename
                                with open(split_nonevul_file, "w+") as nonVulFun:
                                    nonVulFun.write(code)




         except Exception as e:
            traceback.print_exc(file=sys.stdout)
            print("reason", e)
            print("\n commit_id:" + str(commit_id) + "！")
            print("\n index:" + str(index) + "！")
            continue

    df = pd.DataFrame(data)
    df.to_csv('function_with_line.csv', index=False, encoding='utf-8')

    print('completed...')       
    
    print("vul_number",vul_number)

In [132]:
if __name__ == '__main__':
    # delete dir patchAll0206
    if os.path.exists("patchAll0206"):
        shutil.rmtree("patchAll0206")
    # delete dir split0206
    if os.path.exists("split0206"):
        shutil.rmtree("split0206")
    c_cpp_csv = pd.read_csv('all_c_cpp_release2.0.csv', nrows=1,encoding='utf-8')
    # c_cpp_csv = pd.read_csv('all_c_cpp_release2.0.csv',encoding='utf-8')
    result=generate_line_diff(c_cpp_csv)


Processing function with line extraction...: 100%|██████████| 1/1 [00:00<00:00,  1.06it/s]

line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before string->space = 1;
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before string->space *= 2;
line_before 
line_before 
line_before 
line_before if (string->space < 0)
line_before 
line_before 
line_before g_warning ("glyph string length overflows maximum integer size, truncated");
line_before 
line_before new_len = string->space = G_MAXINT - 8;
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_before 
line_be